In [1]:
import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

DEBUG = False
EPOCHS = 10


from baseline_curricula import upload, split_into_epochs, repeat_for_ten_epochs

In [2]:
import config

In [4]:
from baseline_curricula import validate_training_duration_limitation
from itertools import product
import config 

[validate_training_duration_limitation(dataset_name, "llama"+curriculum_name) for dataset_name, curriculum_name in product(config.datasets, config.influence_curricula)]



llama_influence_decr_bins_dirac.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

llama_influence_incr_bins_lognorm.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

llama_influence_incr_cp_dirac.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

llama_influence_decr_cp_dirac.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

llama_influence_top_50_cp_shuffled.pt:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

llama_influence_tracin_sandwich.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [3]:
from baseline_curricula import get_perplexity_ngram

for dataset_name in config.datasets:
    perplexity_df = get_perplexity_ngram(dataset_name)
    upload("perplexity_increasing.pt", repeat_for_ten_epochs(perplexity_df["perplexity"].sort_values(ascending=True).index.to_list()), dataset_name)


./difficulty_ngram/1/babylm_2024_10m_curriculum
Skipping ./difficulty_ngram/1/babylm_2024_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


./difficulty_ngram/1/stratified_equitoken_10m_curriculum
Skipping ./difficulty_ngram/1/stratified_equitoken_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


./difficulty_ngram/1/stratified_10m_curriculum
Skipping ./difficulty_ngram/1/stratified_10m_curriculum, already calculated


  0%|          | 0/1 [00:00<?, ?it/s]

perplexity_increasing.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

In [4]:
from baseline_curricula import get_mattr


for dataset_name in config.datasets:
    mattr_df = get_mattr(dataset_name, n_proc=128)
 #   upload("perplexity_decreasing.pt", split_into_epochs(perplexity_df["perplexity"].sort_values(ascending=False).index.to_list()), dataset)
    upload("mattr_increasing.pt", repeat_for_ten_epochs(mattr_df["mattr"].sort_values(ascending=True).index.to_list()), dataset_name)


./difficulty_mattr/babylm_2024_10m_curriculum
Skipping ./difficulty_mattr/babylm_2024_10m_curriculum, already calculated


No files have been modified since last commit. Skipping to prevent empty commit.


./difficulty_mattr/stratified_equitoken_10m_curriculum
Skipping ./difficulty_mattr/stratified_equitoken_10m_curriculum, already calculated


  0%|          | 0/1 [00:00<?, ?it/s]

mattr_increasing.pt:   0%|          | 0.00/8.00M [00:00<?, ?B/s]

./difficulty_mattr/stratified_10m_curriculum
Skipping ./difficulty_mattr/stratified_10m_curriculum, already calculated


  0%|          | 0/1 [00:00<?, ?it/s]

mattr_increasing.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

In [5]:
from baseline_curricula import get_mattr
import torch


for dataset_name in config.datasets:
    
    torch.manual_seed(42)
    dataset = load_dataset(dataset_name)
    random_order = [torch.randperm(len(dataset["train"])) for _ in range(0,EPOCHS)]
    
    upload("random.pt", random_order, dataset_name)


No files have been modified since last commit. Skipping to prevent empty commit.


Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/53457 [00:00<?, ? examples/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Generating train split:   0%|          | 0/1070321 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/53457 [00:00<?, ? examples/s]

No files have been modified since last commit. Skipping to prevent empty commit.


In [6]:
from baseline_curricula import validate_training_duration_limitation

In [7]:


word_count = lambda d: {"words" : len(d["text"].split())}

WORDS_PER_STAGE = 2*10000000 # there are 5 stages: 2*10M words = 100M total seen during training
for dataset_name in config.datasets:
    
    
    df = load_dataset(dataset_name)["train"].map(word_count, num_proc=100).to_pandas()
    curriculum = []
    for stage_name, stage_df in df.groupby("stage"):
        print(stage_name)
        tokens_in_stage = 0
        curriculum_stage = []
        
        i = 0
        while tokens_in_stage < WORDS_PER_STAGE: # the dataset assumes each stage is repeated for multiple epochs. we will therefore run out of documents before reaching the alloted number of tokens. just re-shuffle and continue
                                                 #    we double-check the total number of tokens seen trough the curriculum below
            stage_df_shuffled = stage_df.sample(frac=1, random_state=42+i)
            for idx, row in stage_df_shuffled.iterrows():            
                if (tokens_in_stage + row["words"]) <= WORDS_PER_STAGE:
                    curriculum_stage.append(idx)
                    tokens_in_stage += row["words"]
                else:
                    tokens_in_stage += row["words"] # so that outer loop breaks
                    break
                
            print(i,tokens_in_stage)
            i+=1
        curriculum.extend(torch.chunk(torch.tensor(curriculum_stage), 2)) # so that there are 2 epochs per stage
    validate_training_duration_limitation(dataset_name, curriculum)

    upload("source_difficulty.pt", curriculum, dataset_name)


C1: Child Directed Speech
0 2839591
1 5679182
2 8518773
3 11358364
4 14197955
5 17037546
6 19877137
7 20000002
C2: Unscripted Dialogue
0 1079286
1 2158572
2 3237858
3 4317144
4 5396430
5 6475716
6 7555002
7 8634288
8 9713574
9 10792860
10 11872146
11 12951432
12 14030718
13 15110004
14 16189290
15 17268576
16 18347862
17 19427148
18 20000024
C3: Scripted Dialogue
0 2041868
1 4083736
2 6125604
3 8167472
4 10209340
5 12251208
6 14293076
7 16334944
8 18376812
9 20000002
C4: Wiki
0 1453539
1 2907078
2 4360617
3 5814156
4 7267695
5 8721234
6 10174773
7 11628312
8 13081851
9 14535390
10 15988929
11 17442468
12 18896007
13 20000009
C5: Written English
0 2539489
1 5078978
2 7618467
3 10157956
4 12697445
5 15236934
6 17776423
7 20000002


Map (num_proc=100):   0%|          | 0/1179014 [00:00<?, ? examples/s]

total_words_seen loris3/babylm_2024_10m_curriculum 99999795


No files have been modified since last commit. Skipping to prevent empty commit.


Map (num_proc=100):   0%|          | 0/100000 [00:00<?, ? examples/s]

...

C1: Child Directed Speech
0 1999999
1 3999998
2 5999997
3 7999996
4 9999995
5 11999994
6 13999993
7 15999992
8 17999991
9 19999990
10 20000002
C2: Children's Books
0 1999995
1 3999990
2 5999985
3 7999980
4 9999975
5 11999970
6 13999965
7 15999960
8 17999955
9 19999950
10 20000025
C3: Dialogue
0 1999987
1 3999974
2 5999961
3 7999948
4 9999935
5 11999922
6 13999909
7 15999896
8 17999883
9 19999870
10 20000016
C4: Educational
0 1999999
1 3999998
2 5999997
3 7999996
4 9999995
5 11999994
6 13999993
7 15999992
8 17999991
9 19999990
10 20000008
C5: Written English
0 1999945
1 3999890
2 5999835
3 7999780
4 9999725
5 11999670
6 13999615
7 15999560
8 17999505
9 19999450
10 20000054


Map (num_proc=100):   0%|          | 0/1070321 [00:00<?, ? examples/s]

total_words_seen loris3/stratified_10m_curriculum 99999943


  0%|          | 0/1 [00:00<?, ?it/s]

source_difficulty.pt:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

In [ ]:
from baseline_curricula import validate_training_duration_limitation
from itertools import product
import config 

[validate_training_duration_limitation(dataset_name, curriculum_name) for dataset_name, curriculum_name in [product(config.datasets, config.baseline_curricula)]]

